# Medicare Outpatient Hospitals - by Provider and Service

In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np

In [8]:
# Read the CSV file (it's in the same directory as this notebook)
file_path = 'Medicare_OP_Hospitals_by_Provider_and_Service_2023.csv'
df = pd.read_csv(file_path)

print(f'Dataset loaded successfully!')
print(f'Shape: {df.shape[0]} rows, {df.shape[1]} columns')
print(f'\nColumns: {list(df.columns)}')
print(f'\nFirst 5 rows:')
print(df.head())
print(f'\nDataset info:')
print(df.info())

Dataset loaded successfully!
Shape: 116799 rows, 18 columns

Columns: ['Rndrng_Prvdr_CCN', 'Rndrng_Prvdr_Org_Name', 'Rndrng_Prvdr_St', 'Rndrng_Prvdr_City', 'Rndrng_Prvdr_State_Abrvtn', 'Rndrng_Prvdr_State_FIPS', 'Rndrng_Prvdr_Zip5', 'Rndrng_Prvdr_RUCA', 'Rndrng_Prvdr_RUCA_Desc', 'APC_Cd', 'APC_Desc', 'Bene_Cnt', 'CAPC_Srvcs', 'Avg_Tot_Sbmtd_Chrgs', 'Avg_Mdcr_Alowd_Amt', 'Avg_Mdcr_Pymt_Amt', 'Outlier_Srvcs', 'Avg_Mdcr_Outlier_Amt']

First 5 rows:
   Rndrng_Prvdr_CCN            Rndrng_Prvdr_Org_Name         Rndrng_Prvdr_St  \
0             10001  Southeast Health Medical Center  1108 Ross Clark Circle   
1             10001  Southeast Health Medical Center  1108 Ross Clark Circle   
2             10001  Southeast Health Medical Center  1108 Ross Clark Circle   
3             10001  Southeast Health Medical Center  1108 Ross Clark Circle   
4             10001  Southeast Health Medical Center  1108 Ross Clark Circle   

  Rndrng_Prvdr_City Rndrng_Prvdr_State_Abrvtn  Rndrng_Prvdr_State_FIP

In [9]:
def clean_medicare_outpatient_by_provider_apc(df, out_csv='medicare_outpatient_by_provider_apc_cleaned.csv'):
    """Clean the Medicare Outpatient Hospitals - by Provider and APC dataset."""
    # Step 1: Rename columns
    column_mapping = {
        'Rndrng_Prvdr_CCN': 'provider_ccn',
        'Rndrng_Prvdr_Org_Name': 'provider_org_name',
        'Rndrng_Prvdr_St': 'provider_street_address',
        'Rndrng_Prvdr_City': 'provider_city',
        'Rndrng_Prvdr_State_Abrvtn': 'provider_state_abbreviation',
        'Rndrng_Prvdr_State_FIPS': 'provider_state_fips',
        'Rndrng_Prvdr_Zip5': 'provider_zip5',
        'Rndrng_Prvdr_RUCA': 'provider_ruca_code',
        'Rndrng_Prvdr_RUCA_Desc': 'provider_ruca_description',
        'APC_Cd': 'apc_code',
        'APC_Desc': 'apc_description',
        'Bene_Cnt': 'beneficiary_count',
        'CAPC_Srvcs': 'apc_services',
        'Avg_Tot_Sbmtd_Chrgs': 'average_total_submitted_charges',
        'Avg_Mdcr_Alowd_Amt': 'average_medicare_allowed_amount',
        'Avg_Mdcr_Pymt_Amt': 'average_medicare_payment_amount',
        'Outlier_Srvcs': 'outlier_services',
        'Avg_Mdcr_Outlier_Amt': 'average_medicare_outlier_amount'
    }
    df = df.rename(columns=column_mapping)
    
    # Step 2: Data type conversion
    # String columns
    string_cols = ['provider_ccn', 'provider_state_fips', 'provider_zip5', 
                   'provider_state_abbreviation', 'apc_code']
    for col in string_cols:
        df[col] = df[col].astype(str).str.strip()
    
    # Numeric columns (handle empty strings as NaN)
    numeric_cols = ['beneficiary_count', 'apc_services', 'outlier_services']
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')  # Nullable integer
    
    financial_cols = ['average_total_submitted_charges', 'average_medicare_allowed_amount',
                     'average_medicare_payment_amount', 'average_medicare_outlier_amount']
    for col in financial_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce').round(2)
    
    # Step 3: Clean string columns
    string_clean_cols = ['provider_org_name', 'provider_street_address', 'provider_city',
                        'provider_ruca_description', 'apc_description']
    for col in string_clean_cols:
        df[col] = df[col].str.strip().str.replace(r'\\\/', '/', regex=True)  # Fix escaped slashes
    
    # Step 4: Validation
    # Check for duplicates
    duplicates = df.duplicated(subset=['provider_ccn', 'apc_code']).sum()
    if duplicates > 0:
        print(f"Warning: Found {duplicates} duplicate rows based on provider_ccn and apc_code.")
    
    # Check for invalid values (e.g., negative counts or payments)
    for col in numeric_cols:
        if (df[col] < 0).any():
            print(f"Warning: Negative values found in {col}.")
    for col in financial_cols:
        if (df[col] < 0).any():
            print(f"Warning: Negative values found in {col}.")
    
    # Step 5: Save cleaned dataset
    df.to_csv(out_csv, index=False)
    print(f"\nCleaned dataset saved to {out_csv}")
    print(f"Rows: {len(df)}, Columns: {list(df.columns)}")
    print("\nFirst 3 rows preview:")
    print(df.head(3).to_string(index=False))
    
    return df

# Usage: Assuming df is loaded from fetch_medicare_outpatient_by_provider_apc
df = fetch_medicare_outpatient_by_provider_apc()  # Run fetch first
df_cleaned = clean_medicare_outpatient_by_provider_apc(df)

Fetching data from CMS API with pagination...
API URL: https://data.cms.gov/data-api/v1/dataset/ccbc9a44-40d4-46b4-a709-5caa59212e50/data
Batch size: 1000 rows per request

Fetching page 1 (offset: 0)... Got 1000 rows (Total so far: 1000)
Fetching page 2 (offset: 1000)... Got 1000 rows (Total so far: 2000)
Fetching page 3 (offset: 2000)... Got 1000 rows (Total so far: 3000)
Fetching page 4 (offset: 3000)... Got 1000 rows (Total so far: 4000)
Fetching page 5 (offset: 4000)... Got 1000 rows (Total so far: 5000)
Fetching page 6 (offset: 5000)... Got 1000 rows (Total so far: 6000)
Fetching page 7 (offset: 6000)... Got 1000 rows (Total so far: 7000)
Fetching page 8 (offset: 7000)... Got 1000 rows (Total so far: 8000)
Fetching page 9 (offset: 8000)... Got 1000 rows (Total so far: 9000)
Fetching page 10 (offset: 9000)... Got 1000 rows (Total so far: 10000)
Fetching page 11 (offset: 10000)... Got 1000 rows (Total so far: 11000)
Fetching page 12 (offset: 11000)... Got 1000 rows (Total so far: 1

KeyboardInterrupt: 